## Configuração (Somente quando necessário)

In [ ]:
! pip install pymysql;

## Restaurar danco de dados

In [11]:
import pymysql
import os
import time
from datetime import datetime
import subprocess
from concurrent.futures import ThreadPoolExecutor

def listar_databases_atuais(cursor):
    cursor.execute('SHOW DATABASES')
    bancos_sommusgestor = [db[0] for db in cursor.fetchall() if 'sommusgestor' in db[0]]
    return bancos_sommusgestor

def deletar_database(cursor, nome):
    cursor.execute(f"DROP DATABASE {nome}")

def criar_database(cursor, nome):
    if(nome.startswith("sommusgestor") and not nome.endswith("_old")):
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nome}")

def restaurar(banco, host, porta, usuario, password, diretorio):
    if(banco.startswith("sommusgestor") and not banco.endswith("_old")):
        comando = f'mysql -h {host} -P {porta} -u {usuario} -p{password} --default-character-set=utf8mb4 {banco} < {diretorio}/{banco}.sql'
        subprocess.run(comando, shell=True, encoding='utf-8')

def tabela_existe(cursor, banco):
    cursor.execute(f"SHOW TABLES LIKE 'versao'")
    resultado = cursor.fetchall()
    existe = bool(resultado)
    if existe:
        print(f"  ⚪ - {banco}")
    return existe

def analisar_final_restauracao(cursor, banco):
    if(banco.startswith("sommusgestor") and banco != "sommusgestor_hangfire" and not banco.endswith("_old")):
        cursor.execute(f"USE {banco}")
        while not tabela_existe(cursor, banco):
            time.sleep(5)

In [12]:
HOST = 'localhost'
PORTA = 3307
USUARIO = 'root'
PASSWORD = 'root'
PASTA = 'backups'
SUBPASTA = '_teste'
DIRETORIO_BACKUP = f'{PASTA}/{SUBPASTA}'

if __name__ == '__main__':
    try:
        with pymysql.connect(host=HOST, user=USUARIO, password=PASSWORD, port=PORTA).cursor() as cursor:
            print(f"🟡 Restauração iniciada: {SUBPASTA}")
            bancos_sommusgestor = listar_databases_atuais(cursor)
            
            print("🔵 [1/4] - Deletando os bancos de dados")
            for banco in bancos_sommusgestor:
                deletar_database(cursor, banco)            

            arquivos_sql_backup = os.listdir(DIRETORIO_BACKUP)

            print("🔵 [2/4] - Criando os databases")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                criar_database(cursor, nome_arquivo_sem_extensao)        

            print("🔵 [3/4] - Restaurando os dados")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                restaurar(nome_arquivo_sem_extensao, HOST, PORTA, USUARIO, PASSWORD, DIRETORIO_BACKUP)

            print("🟠 Aguardando MySql processar backup")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                analisar_final_restauracao(cursor, nome_arquivo_sem_extensao)
            
            print("🔵 [4/4] - Processamento MySql finalizado")

        print(f"✅ Restauração finalizada em {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}")

    except Exception as e:
        print(f'⛔ [Exceção lançada] \n{e}')

 

🟡 Restauração iniciada: _teste
🔵 [1/4] - Deletando os bancos de dados
🔵 [2/4] - Criando os databases
🔵 [3/4] - Restaurando os dados
🟠 Aguardando MySql processar backup
  ⚪ - sommusgestor
  ⚪ - sommusgestor_1
  ⚪ - sommusgestor_809
🔵 [4/4] - Processamento MySql finalizado
✅ Restauração finalizada em 09/01/2024 às 01:49:03
